In [1]:
%cd /home/soda/rcappuzz/work/prepare-data-lakes

/home/soda/rcappuzz/work/prepare-data-lakes


In [2]:
%load_ext autoreload
%autoreload 2


In [3]:
import re
from pathlib import Path

import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import pandas as pd
import polars as pl
import seaborn as sns
from tqdm import tqdm
import os
import polars.selectors as cs
import random

from sklearn.utils import murmurhash3_32

from src import utils

sns.set_theme("paper", style="whitegrid")

In [4]:
cfg = pl.Config()
cfg.set_fmt_str_lengths(150)

polars.config.Config

In [5]:
yago_path = Path("/storage/store3/work/jstojano/yago3/")
facts1_path = Path(yago_path, "facts_parquet/yago_updated_2022_part1")
facts2_path = Path(yago_path, "facts_parquet/yago_updated_2022_part2")
fname = "yagoTypes"
yagotypes_path = Path(facts1_path, f"{fname}.tsv.parquet")
df_types = utils.import_from_yago(yagotypes_path)
fname = "yagoFacts"
yagofacts_path = Path(facts2_path, f"{fname}.tsv.parquet")
yagofacts = utils.import_from_yago(yagofacts_path)
fname = "yagoLiteralFacts"
yagoliteralfacts_path = Path(facts2_path, f"{fname}.tsv.parquet")
yagoliteralfacts = utils.import_from_yago(yagoliteralfacts_path)
fname = "yagoDateFacts"
yagodatefacts_path = Path(facts2_path, f"{fname}.tsv.parquet")
yagodatefacts = utils.import_from_yago(yagodatefacts_path)
yagofacts = yagofacts.drop("num_object")
yagodatefacts = (
    yagodatefacts.with_columns(
        pl.col("cat_object")
        .str.split("^^")
        .list.first()
        .str.to_datetime(strict=False)
        .dt.date()
        .cast(pl.Utf8)
        .alias("cat_object")
    )
    .drop_nulls("cat_object")
    .drop("num_object")
)
yagoliteralfacts = yagoliteralfacts.with_columns(
    pl.when(pl.col("num_object").is_not_null())
    .then(pl.col("num_object"))
    .otherwise(pl.col("cat_object"))
    .alias("cat_object")
).drop("num_object")
df_facts = pl.concat([yagofacts, yagoliteralfacts, yagodatefacts]).drop("id")

In [6]:
wikilabels_path = Path(
    yago_path, "facts_parquet/yago_updated_2022_part2/wikipediaLabels.tsv.parquet"
)
wikipedia_labels = utils.import_from_yago(wikilabels_path)

In [7]:
wikipedia_labels.filter(~pl.col("cat_object").str.ends_with("eng"))

id,subject,predicate,cat_object,num_object
str,str,str,str,f64


In [8]:
for p in Path(yago_path, "facts_parquet/yago_updated_2022_part2/").iterdir():
    if p.stem.startswith("_"):
        continue
    print(p.stem)
    display(utils.import_from_yago(p, debug=False, limit=5))

mappedGeonamesData.tsv


id,subject,predicate,cat_object,num_object
str,str,str,str,f64
null,"""<Andorra>""","""<hasNeighbor>""","""<Spain>""",null
null,"""<Andorra>""","""<hasNeighbor>""","""<Saint_Barthélemy>""",null
null,"""<United_Arab_Emirates>""","""<hasNeighbor>""","""<Saudi_Arabia>""",null
null,"""<United_Arab_Emirates>""","""<hasNeighbor>""","""<Oman>""",null
null,"""<Islamic_Republic_of_Afghanistan>""","""<hasNeighbor>""","""<Turkmenistan>""",null
null,"""<Islamic_Republic_of_Afghanistan>""","""<hasNeighbor>""","""<Mainland_China>""",null
null,"""<Islamic_Republic_of_Afghanistan>""","""<hasNeighbor>""","""<Iran>""",null
null,"""<Islamic_Republic_of_Afghanistan>""","""<hasNeighbor>""","""<Tajikistan>""",null
null,"""<Islamic_Republic_of_Afghanistan>""","""<hasNeighbor>""","""<Pakistan>""",null


mappedGeonamesSources.tsv


id,subject,predicate,cat_object,num_object
str,str,str,str,f64
null,"""<id_?MI25va2AL_w!U_AqwMF9CUpz>""","""<extractionSource>""","""http://geonames.org""",null
null,"""<id_?MI25va2AL_w!U_nKuqN2rIYS>""","""<extractionSource>""","""http://geonames.org""",null
null,"""<id_OafBfbdhRk_w!U_p4CMB4JvPu>""","""<extractionSource>""","""http://geonames.org""",null
null,"""<id_OafBfbdhRk_w!U_H3zOwfFy0!>""","""<extractionSource>""","""http://geonames.org""",null
null,"""<id_WWcsGpC60A_w!U_d3v2ih6jEO>""","""<extractionSource>""","""http://geonames.org""",null
null,"""<id_WWcsGpC60A_w!U_!8xZU!sh5p>""","""<extractionSource>""","""http://geonames.org""",null
null,"""<id_WWcsGpC60A_w!U_rgH!TB0SAr>""","""<extractionSource>""","""http://geonames.org""",null
null,"""<id_WWcsGpC60A_w!U_b7G9BUzW!9>""","""<extractionSource>""","""http://geonames.org""",null
null,"""<id_WWcsGpC60A_w!U_e1sXEoR8Q2>""","""<extractionSource>""","""http://geonames.org""",null


matchedAttributes_en.tsv


...

id,subject,predicate,cat_object,num_object
str,str,str,str,f64
"""<id_KdEX!y?wiG_wda_WURcLBQM1c>""","""owl:Thing""","""<hasWordnetDomain>""","""<wordnetDomain_factotum>""",null
"""<id_g9psfcitLx_wda_WURcLBQM1c>""","""<wordnet_thing_100002452>""","""<hasWordnetDomain>""","""<wordnetDomain_factotum>""",null
"""<id_Vai9oEZk!D_wda_WURcLBQM1c>""","""<wordnet_nothing_113740168>""","""<hasWordnetDomain>""","""<wordnetDomain_factotum>""",null
"""<id_r9CiCKIlsX_wda_WURcLBQM1c>""","""<wordnet_whole_100003553>""","""<hasWordnetDomain>""","""<wordnetDomain_factotum>""",null
"""<id_k1DTqzDW2q_wda_WURcLBQM1c>""","""<wordnet_living_thing_100004258>""","""<hasWordnetDomain>""","""<wordnetDomain_factotum>""",null
"""<id_!mzSYbE2m?_wda_WURcLBQM1c>""","""<wordnet_organism_100004475>""","""<hasWordnetDomain>""","""<wordnetDomain_factotum>""",null
"""<id_jx0Z!32iCA_wda_WURcLBQM1c>""","""<wordnet_benthos_100005787>""","""<hasWordnetDomain>""","""<wordnetDomain_factotum>""",null
"""<id_FcbnkDHJs4_wda_WURcLBQM1c>""","""<wordnet_heterotroph_100006024>""","""<hasWordnetDomain>""","""<wordnetDomain_factotum>""",null
"""<id_NGRL!wmn5g_wda_WURcLBQM1c>""","""<wordnet_life_100006269>""","""<hasWordnetDomain>""","""<wordnetDomain_factotum>""",null


yagoWordnetIds.tsv


id,subject,predicate,cat_object,num_object
str,str,str,f64,f64
null,"""owl:Thing""","""<hasSynsetId>""",1.0000174e8,null
null,"""<wordnet_physical_entity_100001930>""","""<hasSynsetId>""",1.0000193e8,null
null,"""<wordnet_abstraction_100002137>""","""<hasSynsetId>""",1.00002137e8,null
null,"""<wordnet_thing_100002452>""","""<hasSynsetId>""",1.00002452e8,null
null,"""<wordnet_object_100002684>""","""<hasSynsetId>""",1.00002684e8,null
null,"""<wordnet_whole_100003553>""","""<hasSynsetId>""",1.00003553e8,null
null,"""<wordnet_congener_100003993>""","""<hasSynsetId>""",1.00003993e8,null
null,"""<wordnet_living_thing_100004258>""","""<hasSynsetId>""",1.00004258e8,null
null,"""<wordnet_organism_100004475>""","""<hasSynsetId>""",1.00004475e8,null


peopleByCategorySource.tsv


id,subject,predicate,cat_object,num_object
str,str,str,str,f64
"""<id_6LLYDjxmjv_Tm9_NeK?6DKz1Q>""","""<id_TkIEqw6ZnR_KCM_9o9YmDSiad>""","""<extractionSource>""","""<wikicat_Living_people>""",null
null,"""<id_6LLYDjxmjv_Tm9_NeK?6DKz1Q>""","""<extractionTechnique>""","""PeopleExtractor""",null
"""<id_1G0BhY0VMg_Tm9_vEHHcUz6jq>""","""<id_wGHfubCwBs_KCM_9o9YmDSiad>""","""<extractionSource>""","""<wikicat_1794_deaths>""",null
null,"""<id_1G0BhY0VMg_Tm9_vEHHcUz6jq>""","""<extractionTechnique>""","""PeopleExtractor""",null
"""<id_o3xqsaGJqJ_Tm9_hrio6MjfwR>""","""<id_EcFSy3pfWg_KCM_9o9YmDSiad>""","""<extractionSource>""","""<wikicat_1941_births>""",null
null,"""<id_o3xqsaGJqJ_Tm9_hrio6MjfwR>""","""<extractionTechnique>""","""PeopleExtractor""",null
"""<id_vSGxVP26zi_Tm9_xkmK570HXL>""","""<id_TL?FW9ny9s_KCM_9o9YmDSiad>""","""<extractionSource>""","""<wikicat_1885_deaths>""",null
null,"""<id_vSGxVP26zi_Tm9_xkmK570HXL>""","""<extractionTechnique>""","""PeopleExtractor""",null
"""<id_tRjQu8h?JJ_Tm9_EurvlOlvy0>""","""<id_WuVXfXU4S!_KCM_9o9YmDSiad>""","""<extractionSource>""","""<wikicat_1991_births>""",null


In [11]:
p = Path(
    yago_path, "facts_parquet/yago_updated_2022_part2/yagoSchema.tsv.parquet"
)

utils.import_from_yago(p, debug=False, limit=1000)

id,subject,predicate,cat_object,num_object
str,str,str,str,f64
"""<id_E6n!IJn!Jr_O1n_0EsGYjWbi9>""","""<graduatedFrom>""","""rdfs:range""","""<wordnet_university_108286569>""",null
"""<id_?RGufJFBdR_O1n_uRZHKm6aLD>""","""<hasGivenName>""","""rdfs:range""","""xsd:string""",null
"""<id_EGcgvQQgMO_O1n_KukFM4VlQ7>""","""<_yagoMetadata>""","""rdfs:range""","""xsd:String""",null
"""<id_AypJzCSjLb_O1n_KdEX!y?wiG>""","""<hasCurrency>""","""rdfs:range""","""owl:Thing""",null
"""<id_?!VtKe8N6Z_O1n_uRZHKm6aLD>""","""<hasGeonamesClassId>""","""rdfs:range""","""xsd:string""",null
"""<id_e!1Dxw5lnJ_O1n_hrUs8UA4yf>""","""<dealsWith>""","""rdfs:range""","""<wordnet_country_108544813>""",null
"""<id_iqvUKl!ECA_O1n_uRZHKm6aLD>""","""<hasMotto>""","""rdfs:range""","""xsd:string""",null
"""<id_w!UqetLWZe_O1n_FNZ17GW0Jo>""","""<hasNeighbor>""","""rdfs:range""","""<yagoGeoEntity>""",null
"""<id_yqifFULyde_O1n_CkqeGfgJ0O>""","""<hasGDP>""","""rdfs:range""","""<yagoMonetaryValue>""",null


# Full tables

In [23]:
subjects_with_wordnet = (
    df_types.filter(pl.col("cat_object").str.starts_with("<wordnet_"))
    .select(pl.col("subject"), pl.col("cat_object"))
    .rename({"cat_object": "type"})
)
n_groups = len(subjects_with_wordnet.unique("type"))

In [25]:
def clean_string(string_to_clean):
    pattern = re.compile(r"<{1}([a-zA-Z0-9_]+)>{1}")
    m = re.sub(pattern, "\\1", string_to_clean)
    return m

In [6]:
max_fields = 2
dest_path = Path("data/yago3-dl/wordnet_vldb")

In [10]:
for this_type, this_df in tqdm(subjects_with_wordnet.group_by("type"), total=n_groups):
    clean_type = clean_string(this_type)
    joined_df = df_facts.join(this_df.select(pl.col("subject", "type")), on="subject")
    if len(joined_df) == 0:
        continue
    base_df = joined_df.select(pl.col("subject").unique()).lazy()
    for idx, grp in joined_df.group_by(by=["predicate"]):
        this_predicate = clean_string(idx[0])
        grp = (
            grp.group_by("subject")
            .head(max_fields)
            .select(pl.col("subject", "cat_object"))
            .rename({"cat_object": this_predicate})
        )
        try:
            # Try to convert column to numerical
            grp = grp.with_columns(pl.col(this_predicate).cast(pl.Float64))
        except pl.ComputeError:
            # Column is not numerical, do nothing
            pass
        grp = grp.lazy()
        base_df = base_df.join(grp, on="subject", how="left")
    base_df = base_df.rename({"subject": clean_type})
    df_name = f"wordnet_vldb-{clean_type}.parquet"
    base_df = base_df.collect()
    if len(base_df) > 0:
        base_df.write_parquet(Path(dest_path, df_name))

100%|██████████| 1015/1015 [01:59<00:00,  8.50it/s]


## Profile base tables

In [5]:
max_fields = 2
dest_path = Path("data/yago3-dl/wordnet_vldb")

In [10]:
list_stats = []
for path in Path(dest_path).glob("*.parquet"):
    this_df = pl.read_parquet(path)
    rows, cols = this_df.shape
    n_num = this_df.select(cs.numeric()).shape[1]
    c_num = this_df.select(~cs.numeric()).shape[1]

    list_stats.append(
        {
            "rows": rows,
            "cols": cols,
            "n_num": n_num,
            "c_num": c_num,
            "size": path.stat().st_size
        }
    )
df_stats = pl.from_dicts(list_stats)

In [17]:
df_stats.filter((pl.col("rows") > 100) & (pl.col("cols") > 8)).with_columns(
    (pl.col("rows")*pl.col("cols")).alias("cells"),
).with_columns(
    (pl.col("size")/pl.col("cells")).alias("avg_size")
).select(pl.col("avg_size").mean())

avg_size
f64
2.08772


In [20]:
220*9

4760000.0

In [28]:
df_stats.filter(pl.col("cols") > 8).select(
    pl.col("rows").median(),
    pl.col("cols").median(),
    pl.len()
)

rows,cols,len
f64,f64,u32
1602.0,14.0,277


In [30]:
gen_byes=(277 * 1600 * 14 + 500 * 300 * 8)*2

In [35]:
(277+500) * 135

104895

In [34]:
2e9//gen_byes

135.0

In [14]:
df_stats.select(pl.col("cols").mean().alias("mean"),pl.col("cols").median().alias("median"),)

mean,median
f64,f64
8.195627,6.0


## Build subtables

In [19]:
def generate_table(table, col_comb, subject):
    selected_ = subject + list(col_comb)
    new_df = table.select(selected_).filter(
        pl.any_horizontal(pl.col(col_comb).is_not_null())
    )
    return new_df

In [22]:
def generate_batch(
    target_columns,
    base_table,
    table_name,
    dest_path,
    subject,
    case,
    resample=False,
    sample_fraction=0.7,
    n_resample=2, 
    min_occurrences=100,
    limit_break=50,
    gen_break=10,
):
    dest_path = Path(base_path, table_name)
    break_counter = 0
    gen_counter = 0
    good_comb = set()
    bad_comb = set()
    while break_counter < limit_break and gen_counter < gen_break:
        comb = random.sample(target_columns, k=random.randint(1, len(target_columns)))
        comb = tuple(comb)
        if (comb not in good_comb) and (comb not in bad_comb):
            table = generate_table(base_table, comb, subject)
            if len(table) > min_occurrences:
                good_comb.add(comb)
                gen_counter += 1
                fname = (
                    table_name
                    + "-"
                    + str(murmurhash3_32("-".join(comb[:])))
                    + case
                    + ".parquet"
                )
                destination_path = Path(dest_path, fname)
                table.write_parquet(destination_path)
                if resample:
                    for sample_counter in range(n_resample):
                        resampled_table = table.sample(fraction=sample_fraction)
                        fname = (
                            table_name
                            + "-"
                            + str(murmurhash3_32("-".join(comb[:])))
                            + "-"
                            + str(sample_counter)
                            + "-"
                            + case
                            + ".parquet"
                        )
                        destination_path = Path(dest_path, fname)
                        resampled_table.write_parquet(destination_path)
            else:
                bad_comb.add(comb)
                break_counter += 1
        else:
            break_counter += 1

In [50]:
base_path = Path("data/yadl/wordnet_vldb")
comb_size = 2
min_occurrences = 50
total_ = sum(1 for _ in base_path.glob("*.parquet"))
limit_break = 100
gen_break = 50
n_resample = 2
sample_fraction = 0.7
resample = False

bc = 0


In [ ]:
for pth in tqdm(base_path.glob("*.parquet"), total=total_):
    table_name = pth.stem

    tgt_table = pl.read_parquet(pth)
    n_rows, n_cols = tgt_table.shape
    n_cols -= 1  # Not counting column `subject`

    subject = [tgt_table.columns[0]]
    target_columns = tgt_table.columns[1:]

    num_columns = tgt_table.select(cs.numeric()).columns
    cat_columns = [_ for _ in tgt_table.select(~cs.numeric()).columns if _ != "subject"]

    if n_rows == 0:
        continue

    dest_path = Path(base_path, table_name)
    os.makedirs(dest_path, exist_ok=True)

    # # All columns
    generate_batch(
        target_columns,
        tgt_table,
        table_name,
        subject,
        "",
        min_occurrences,
        limit_break=limit_break,
        gen_break=gen_break,
    )

    # Numerical columns
    if len(num_columns) >= 2:
        # print(table_name)
        generate_batch(
            num_columns,
            tgt_table,
            subject,
            "num",
            min_occurrences,
            limit_break=limit_break,
            gen_break=gen_break,
        )